## THE ADVISORY FILE
#mock data

In [ ]:
import pandas as pd
import numpy as np

crops = ['Rice', 'Maize', 'Coffee', 'Cotton', 'Grapes']
data_size = 500

    
crop_seasons = {
    #typical planting months
    'Rice': [3, 4, 5, 10, 11],    # Wet seasons
    'Maize': [3, 4, 9, 10],       # Early rains
    'Coffee': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], # Perennial
    'Cotton': [5, 6, 7, 8],       # Warmer months
    'Grapes': [11, 12, 1, 2]      # Specific cycle
}
data_list = []
for _ in range(1000):
    crop = np.random.choice(list(crop_seasons.keys()))
    month = np.random.choice(crop_seasons[crop])
    
    data_list.append({
        'month': month,
        'N': np.random.randint(0, 140),
        'P': np.random.randint(5, 145),
        'K': np.random.randint(5, 205),
        'temperature': np.random.uniform(15, 40),
        'humidity': np.random.uniform(15, 100),
        'ph': np.random.uniform(4.5, 9.0),
        'rainfall': np.random.uniform(20, 300),
        'label': crop
    })
    
if crop == 'Rice':
    temperature = np.random.uniform(20, 35)
    rainfall = np.random.uniform(120, 300)
    humidity = np.random.uniform(70, 100)
    ph = np.random.uniform(5.0, 7.0)
elif crop == 'Maize':
    temperature = np.random.uniform(18, 32)
    rainfall = np.random.uniform(50, 150)
    humidity = np.random.uniform(50, 80)
    ph = np.random.uniform(5.5, 7.5)
elif crop == 'Coffee':
    temperature = np.random.uniform(15, 28)
    rainfall = np.random.uniform(80, 200)
    humidity = np.random.uniform(60, 90)
    ph = np.random.uniform(6.0, 7.0)
elif crop == 'Cotton':
    temperature = np.random.uniform(25, 38)
    rainfall = np.random.uniform(40, 120)
    humidity = np.random.uniform(40, 70)
    ph = np.random.uniform(6.0, 8.0)
elif crop == 'Grapes':
    temperature = np.random.uniform(15, 30)
    rainfall = np.random.uniform(20, 80)
    humidity = np.random.uniform(40, 60)
    ph = np.random.uniform(5.5, 7.5)
elif temperature < 18 or rainfall < 40 or ph < 5:
    label = 'No Plant'


df= pd.DataFrame(data_list)
df.to_csv('mock_seasonal_crop_data.csv', index= False)
print(df.head)
print('mock dataset created successfully!')

def risk_assessment ( humidity, rainfall, temp):
    risk_score = 0
    factors = []

    #fungal attack is when humidity is high
    if humidity > 80 and 20 <= temp<=30:
        risk_score += 60
        factors.append("Heavy rainfall and warmth: Your crops are at risk of fungal blight.")
    if rainfall>200:
        risk_score += 20
        factors.append("Heavy rainfall: Monitor for water-borne pests.")
    if rainfall <20 and temp >35:
        risk_score +=70
        factors.append("Extreme heat and low rainfall amounts: Your crops are at high risk of wilting")
    #categorixing the risk
    level = "low"
    if risk_score >70 : level = "high"
    elif risk_score > 40: level = 'medium'


    return {"level": level, "score": risk_score, "warnings": factors}


### advisory logic

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib
import datetime

df= pd.read_csv('mock_seasonal_crop_data.csv')
x= df.drop('label', axis=1)
y=df['label']

model =RandomForestClassifier(n_estimators=100) #random forest with 100 decision trees
model.fit(x, y)

def get_farmer_advice(lat, lon, soil_type, n, p, k, ph):
    current_month = datetime.datetime.now().month
    # For now, I will use mock weather values
    mock_temp = 25.5
    mock_humidity = 70.0
    mock_rainfall = 100.0
    #prediction
    prediction = model.predict([[current_month, n, p, k, mock_temp, mock_humidity, ph, mock_rainfall]])
    month_name= datetime.datetime.now().strftime("%B")

    risk_info = risk_assessment(temperature, humidity, rainfall)

    advice = {
        "location": f"Coordinates ({lat}, {lon})",
        "recommended_crop": prediction[0],
        "risk_level": risk_info["level"],
        "soil_analysis": f"Type: {soil_type}, PH: {ph}",
       
        "alerts": risk_info["warnings"],
        "message":f"We are currently in the month of {month_name} and Based on your {soil_type} soil and current climate, we recommend planting {prediction[0]}, however, be ware of {risk_info['level']} risk levels."
    }
    return advice
print(get_farmer_advice(-1.28, 36.82, "loamy", 90, 40, 40, 6.5))
    